In [11]:
import pandas as pd
import time
from itertools import combinations
import math

# db1=pd.read_excel('/content/20_1_Transactions.xlsx')
db1=pd.read_csv('/content/Database_1.csv',header=None)
print(db1)
print()

db1_l=db1.values.tolist()
db1_list=[]
for i in db1_l:
  l=[]
  for j in i:
    if type(j)==str:
      j=j.strip()
      l.append(j)
  db1_list.append(l)

print("Transactions Without Redundancy:")
for i in db1_list:
  print(i)
print()

total_transactions=len(db1_list)

min_support=float(input("Minimum Support: "))
min_confidence=float(input("Minimum Confidence: "))
print()

start_time=time.time()

s=20*(min_support/100)
size=0
ans=[]
flist=[]
items=[]

def freq(list_l):
  if size==0:
    f={}
    for i in list_l:
      for j in i:
        if j not in f.keys():
          f[j]=1
          items.append(j)
        else:
          f[j]=f[j]+1
  elif size>0:
    f={}
    for i in list_l:
      if i not in f.keys():
        f[i]=1
      else:
        f[i]+=1
  flist.append(f)

def checkminsup(list_l):
  fl=[]
  freq(list_l)
  for i in flist[size].keys():
    if flist[size][i] >= s: #Stores min support items after computation of freq in fl
      fl.append(i)
  ans.append(fl) #fl items are stored in ans list

def unique_items(list_1):
  ll=[]
  if size==1:
    ll=list_1
  else:
    for i in list_1:
      for j in i:
        if j not in ll:
          ll.append(j)
  return ll

def itemsupport(total_transaction = 0, matching_transaction = 0):
    return ((matching_transaction/total_transaction)*100)

def itemconfidence(matching_itemset = 0, matching_left_item = 0):
    return((matching_itemset/matching_left_item)*100)

if size==0:
  checkminsup(db1_list)
  size+=1

while len(ans[-1])>0:
  newl1=unique_items(ans[size-1])
  newl1=combinations(newl1,size+1)
  newl1=list(newl1)
  fl=[]
  for i in newl1:
    for j in db1_list:
      i=frozenset(i)
      j=frozenset(j)
      condition=i.issubset(j)
      if condition:
        fl.append(i)
  checkminsup(fl)
  size+=1

for i in range(len(ans)-1):
  print("Itemset of size",i+1,end=":\n")
  if i==0:
    for j in ans[i]:
      fval=flist[i][j]
      print("{",end="")
      print(j,end="")
      print("}:",fval)
  else:
    for j in ans[i]:
      fval=flist[i][j]
      k=list(j)
      print("{",end="")
      for l in k[:-1]:
        print(l,end=", ")
      print(k[-1],end="}: ")
      print(fval)
  print()

fans=[]
for i in range(len(ans)):
  if i==0:
    for j in ans[i]:
      fans.append([j])
  else:
    for j in ans[i]:
      k=list(j)
      fans.append(k)

rules={}
for candidates in fans:
  if(len(candidates))<2:
    continue
  for i in range(1,len(candidates)):
    LHS=[]
    RHS=[]
    cii=[]

    for j in range(i):
      cii.append(j)
    
    for j in cii:
      LHS.append(candidates[j])
    
    for j in range(len(candidates)):
      if j not in cii:
        RHS.append(candidates[j])
    
    call=0
    cleft=0
    for element in range(len(db1_list)):
      if(all(data in db1_list[element] for data in LHS)):
        cleft+=1
        if(all(data in db1_list[element] for data in RHS)):
          call+=1
    
    if(itemconfidence(itemsupport(total_transactions,call), itemsupport(total_transactions,cleft))) >= min_confidence:
      key=""
      value=""
      conf=round(itemconfidence(itemsupport(total_transactions,call),itemsupport(total_transactions,cleft)),2)

      for j in LHS:
        key=key+" "+j
      
      key=key.strip()

      for j in RHS:
        value=value+" "+j
      
      value.strip()

      for key in rules.keys():
        rules[key].append([value,conf])
      
      else:
        temp=[]
        temp.append([value,conf])
        rules[key]=temp
    
    lip=len(cii)-1
    item_comb=math.factorial(len(candidates))/math.factorial(i)

    while(item_comb>0):
      LHS=[]
      RHS=[]

      for j in range(i-1,-1,-1):
        if(cii[j]<(len(candidates))-(lip-j)-1):
          temp_pos=cii[j]+1

          for k in range(j,i):
            cii[k]=temp_pos
            temp_pos+=1
          break
        else:
          continue
      
      for j in cii:
        LHS.append(candidates[j])
      
      for j in range(len(candidates)):
        if j not in cii:
          RHS.append(candidates[j])
      
      call=0
      cleft=0
      for element in range(len(db1_list)):
        if(all(data in db1_list[element] for data in LHS)):
          cleft+=1
          if(all(data in db1_list[element] for data in RHS)):
            call+=1
      
      if(itemconfidence(itemsupport(total_transactions,call),itemsupport(total_transactions,cleft))<min_confidence):
        item_comb-=1
        continue
      
      key=""
      value=""
      for j in LHS:
        key=key+" "+j
      key=key.strip()

      for j in RHS:
        value=value+" "+j
      value=value.strip()

      conf=round(itemconfidence(itemsupport(total_transactions,call),itemsupport(total_transactions,cleft)),2)

      if key in rules.keys():
        if (value not in [item[0] for item in rules[key]]):
          rules[key].append([value,conf])
      
      else:
        temp=[]
        temp.append([value,conf])
        rules[key]=temp

      item_comb-=1
  
print("All the possible association rules along with their confidence are as follows:")
for key in rules:
  for value in rules[key]:
    print("{",end="")
    for j in key:
      if j==" ":
        print(", ",end="")
      else:
        print(j,end="")
    print("} -> {",end="")
    for j in range(len(value[0])):
      if value[0][j]==" ":
        if j==0:
          continue
        else:
          print(end=", ")
      else:
        print(value[0][j],end="")
    print("}, Confidence: " + str(value[1]) + "%")
   # print("{" + key + "} -> {" + value[0] + "} " + str(value[1]))

total_time=time.time()-start_time
print("Total Time taken for computation:",total_time)

          0         1          2         3          4       5       6   \
0       Ball  Biscuits        Pan       NaN        NaN     NaN     NaN   
1       Book     Glass     Cheese       Bag        NaN     NaN     NaN   
2       Milk       Oil     Sandal    Cheese     Orange     NaN     NaN   
3       Bowl    Gloves       Shoe       NaN        NaN     NaN     NaN   
4        Net       Pen        NaN       NaN        NaN     NaN     NaN   
5     Pillow     Water      Apple  Biscuits     Carrom    Ball  Sandal   
6    Antenna      Ball       Bowl     Glass      Juice     Oil     Pen   
7        Bag      Book    Blender      Bulb        Egg    Milk     Net   
8      Apple      Ball   Biscuits      Book       Bulb  Cheese     Egg   
9        Bag  Biscuits       Bowl    Cheese        NaN     NaN     NaN   
10     Glass      Meat        Net       Pan  Powerbank     NaN     NaN   
11   Antenna     Apple        Bat  Biscuits        NaN     NaN     NaN   
12      Bowl      Bulb    Coolers     